In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [36]:
import pandas as pd
import geopandas as gpd
import maup
import time

In [37]:
# from class
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial

In [38]:
maup.progress.enabled = True

ct_df = gpd.read_file("./CT/CT.shp")

print(ct_df)

    STATEFP20 COUNTYFP20              NAME20  G20PRED  G20PRER  TOTPOP  \
0          09        011    Montville 004-00     1486     1547  6089.0   
1          09        005  Barkhamsted 001-00     1154     1228  3647.0   
2          09        005    Bethlehem 001-00      937     1393  3385.0   
3          09        005  Bridgewater 001-00      667      549  1662.0   
4          09        005       Canaan 001-00      437      209  1080.0   
..        ...        ...                 ...      ...      ...     ...   
736        09        009    Waterbury 071-03     1536     1528  8128.0   
737        09        009    New Haven 001-02      160        8  2336.0   
738        09        009    New Haven 007-02     2133      192  6867.0   
739        09        009    New Haven 002-02     1150      103  3083.0   
740        09        009    New Haven 022-01      489       38  4088.0   

       HISP  NH_WHITE  NH_BLACK  NH_AMIN  ...    HVAP    WVAP   BVAP  AMINVAP  \
0     461.0    4805.0     208.

In [39]:
# See all column names
print(ct_df.columns)

Index(['STATEFP20', 'COUNTYFP20', 'NAME20', 'G20PRED', 'G20PRER', 'TOTPOP',
       'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI',
       'NH_OTHER', 'NH_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP',
       'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP', 'CD', 'geometry'],
      dtype='object')


# Partition and updaters

In [40]:
graph = Graph.from_geodataframe(ct_df)

# updaters
updaters = {
    "cut_edges": cut_edges,
    "population": Tally("TOTPOP", alias="population"),
}

# Define the election object
election = Election("2020_Presidential", {"Democratic": "G20PRED", "Republican": "G20PRER"})

# Define your updaters dictionary
updaters = {"population": Tally("TOTPOP", alias="population"), "2020_Presidential": election}

# Create an initial partition
initial_partition = Partition(graph, assignment="CD", updaters=updaters)

# Efficiency Gap Calculation

In [42]:
eff_gap = initial_partition["2020_Presidential"].efficiency_gap()

print(f"Efficiency Gap: {eff_gap}")

Efficiency Gap: 0.2960990182384431


# Mean Median Difference Calculation

In [43]:
mean_median_diff = initial_partition["2020_Presidential"].mean_median()

print(f"Mean-Median Difference: {mean_median_diff}")

Mean-Median Difference: -0.0008308592017549632


# Democratic vs Republican Won Districts

In [47]:
# The ElectionResults object for "2020_Presidential"
election_results = initial_partition["2020_Presidential"]

# Use the wins method to get the number of districts won by each party
dem_won_districts = election_results.wins("Democratic")
rep_won_districts = election_results.wins("Republican")

# Print out the counts
print(f"Democratic-won Districts: {dem_won_districts}")
print(f"Republican-won Districts: {rep_won_districts}")

Democratic-won Districts: 5
Republican-won Districts: 0


# Number of Cut Edges

In [48]:
# Ccut edges can be accessed from the partition object
number_of_cut_edges = len(initial_partition["cut_edges"])

print(f"Number of cut edges: {number_of_cut_edges}")

Number of cut edges: 205
